<a href="https://colab.research.google.com/github/michaelscheinfeild/pytorch-transformer/blob/main/InferenceA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# transformer

transformer from scratch
https://www.youtube.com/watch?v=ISNdQcPhsts
https://github.com/hkproj/pytorch-transformer


In [1]:
#!git clone https://github.com/hkproj/pytorch-transformer.git
!git clone https://github.com/michaelscheinfeild/pytorch-transformer.git

Cloning into 'pytorch-transformer'...
remote: Enumerating objects: 146, done.
remote: Counting objects: 100% (53/53), done.
remote: Compressing objects: 100% (15/15), done.
remote: Total 146 (delta 45), reused 38 (delta 38), pack-reused 93
Receiving objects: 100% (146/146), 5.06 MiB | 5.47 MiB/s, done.
Resolving deltas: 100% (82/82), done.


In [2]:
%cd pytorch-transformer


/content/pytorch-transformer


In [3]:
%ls

attention_visual.ipynb  config.py         Local_Train.ipynb  train.py
Beam_Search.ipynb       dataset.py        model.py
Colab_TrainA.ipynb      InferenceA.ipynb  README.md
Colab_Train.ipynb       Inference.ipynb   requirements.txt


In [4]:
from pathlib import Path
import torch
import torch.nn as nn


In [5]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 11.5 MB/s eta 0:00:00


In [6]:
!pip install tokenizers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 12.3 MB/s eta 0:00:00


In [7]:
!pip install torchmetrics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 763.4/763.4 kB 7.3 MB/s eta 0:00:00


In [8]:
from config import get_config, get_weights_file_path

In [9]:
from train import get_model, get_ds, run_validation

In [10]:
# Define the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

Using device: cuda


In [11]:
config = get_config()
config

{'batch_size': 8,
 'num_epochs': 20,
 'lr': 0.0001,
 'seq_len': 350,
 'd_model': 512,
 'lang_src': 'en',
 'lang_tgt': 'it',
 'model_folder': 'weights',
 'model_basename': 'tmodel_',
 'preload': None,
 'tokenizer_file': 'tokenizer_{0}.json',
 'experiment_name': 'runs/tmodel'}

In [12]:
train_dataloader, val_dataloader, tokenizer_src, tokenizer_tgt = get_ds(config)

Generating train split:   0%|          | 0/32332 [00:00<?, ? examples/s]

Max length of source sentence: 309
Max length of target sentence: 274


In [13]:
print(train_dataloader.dataset)

In [14]:
print(tokenizer_src.get_vocab_size())

15698


In [15]:
model = get_model(config, tokenizer_src.get_vocab_size(), tokenizer_tgt.get_vocab_size()).to(device)

In [16]:
model

Transformer(
  (encoder): Encoder(
    (layers): ModuleList(
      (0-5): 6 x EncoderBlock(
        (self_attention_block): MultiHeadAttentionBlock(
          (w_q): Linear(in_features=512, out_features=512, bias=False)
          (w_k): Linear(in_features=512, out_features=512, bias=False)
          (w_v): Linear(in_features=512, out_features=512, bias=False)
          (w_o): Linear(in_features=512, out_features=512, bias=False)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (feed_forward_block): FeedForwardBlock(
          (linear_1): Linear(in_features=512, out_features=2048, bias=True)
          (dropout): Dropout(p=0.1, inplace=False)
          (linear_2): Linear(in_features=2048, out_features=512, bias=True)
        )
        (residual_connections): ModuleList(
          (0-1): 2 x ResidualConnection(
            (dropout): Dropout(p=0.1, inplace=False)
            (norm): LayerNormalization()
          )
        )
      )
    )
    (norm): LayerNormalization

In [17]:
%%capture
from google.colab import drive
drive.mount('/content/drive')

In [18]:
config['model_folder'] = '..//drive/MyDrive/Models/pytorch-transformer/weights'
config['tokenizer_file'] = '..//drive/MyDrive/Models/pytorch-transformer/vocab/tokenizer_{0}.json'
config

{'batch_size': 8,
 'num_epochs': 20,
 'lr': 0.0001,
 'seq_len': 350,
 'd_model': 512,
 'lang_src': 'en',
 'lang_tgt': 'it',
 'model_folder': '..//drive/MyDrive/Models/pytorch-transformer/weights',
 'model_basename': 'tmodel_',
 'preload': None,
 'tokenizer_file': '..//drive/MyDrive/Models/pytorch-transformer/vocab/tokenizer_{0}.json',
 'experiment_name': 'runs/tmodel'}

In [19]:
# Load the pretrained weights
model_filename = get_weights_file_path(config, f"06")#29
model_filename

'../drive/MyDrive/Models/pytorch-transformer/weights/tmodel_06.pt'

In [20]:

state = torch.load(model_filename)
model.load_state_dict(state['model_state_dict'])

<All keys matched successfully>

In [21]:
run_validation(model, val_dataloader, tokenizer_src, tokenizer_tgt, config['seq_len'], device, lambda msg: print(msg), 0, None, num_examples=10)

--------------------------------------------------------------------------------
    SOURCE: At this moment I discover that I forgot to take my parcel out of the pocket of the coach, where I had placed it for safety; there it remains, there it must remain; and now, I am absolutely destitute.
    TARGET: In quel momento mi accorgo che ho dimenticato il mio involto nella sacca della carrozza, ove l'avevo messo perché fosse più sicuro; devo lasciarvelo per forza e io non ho nessuna risorsa, nulla.
 PREDICTED: In quel momento mi accorsi che avevo dimenticato la mia scala , mi accorsi che era stata la porta della carrozza .
--------------------------------------------------------------------------------
    SOURCE: His mind went rapidly over all the arguments in favour of his decision.
    TARGET: Sentiva perfino che non dir nulla, adesso, significava offenderla.
 PREDICTED: Il suo pensiero si fermò in tanto più forte di un affare di cui si era messo a parlare .
----------------------------